# UOV (Unbalanced Oil and Vinegar) Implementation Notebook

This implementation is based on reference code provided by professor John Baena, used with permission. The original version was adapted and extended by Tomás Escobar to include hashing, salt generation, structured key handling, debug logging, and a cleaner class-based interface.

## Overview

This notebook explains the key generation, signing, and verification functions for the UOV digital signature scheme. We'll go through each function step-by-step, understanding what it does and how it implements the UOV specification.

In [30]:
from sage.all import *

## 1. The `Upper()` Function

### Explanation

In UOV, we work with quadratic forms represented as matrices. A quadratic form can be written as x^T · M · x where M is a matrix. However, the same quadratic form can be represented by different matrices-specifically, M and its transpose M^T give the same result.

The `Upper()` function converts any square matrix M into an **upper triangular matrix** that defines the same quadratic form. This is done by:
- Taking the sum of symmetric positions: M[i,j] becomes M[i,j] + M[j,i]
- Clearing the lower triangle: M[j,i] becomes 0

This ensures we have a canonical representation with no redundancy.

In [31]:
def Upper(M):
    """
    input:
    M = square matrix
    
    output:
    Upper triangular matrix that defines the same quadratic form as M
    """
    nn = M.ncols()
    limit = 1
    for i in range(nn):
        for j in range(i+limit, nn):
            M[i, j] += M[j, i]  # Combine symmetric entries
            M[j, i] = 0          # Clear lower triangle
    return M

## 2. The `Eval()` Function

### Explanation

To verify a signature or compute values during signing, we need to evaluate quadratic polynomials. Each polynomial in UOV is represented as a matrix M, and we evaluate it at a vector x using the quadratic form: p(x) = x^T · M · x.

The `Eval()` function takes a list of matrices (representing multiple polynomials) and a vector, then returns the vector of all evaluations.

In [32]:
def Eval(F, x):
    """
    input:
    F = list of square matrices of size len(x)
    x = a vector with entries in F_q
    
    output:
    vector with the evaluation at x of the quadratic forms defined by the matrices in F
    """
    return vector([ x*M*x for M in F])

## 3. The `Get_UOV_Keys()` Function

### Explanation

This is the core key generation function. It implements the key generation algorithm from the UOV specification:

**Step-by-step process:**

1. **Set up the field**: Create the finite field F_q with q elements
2. **Generate the Oil space**: 
   - Sample a random (n-m) × m matrix O
   - Create O_bar = [O | I_m]^T (the Oil space matrix with identity appended)
3. **Generate public key matrices**: For each polynomial i ∈ [m]:
   - Sample upper triangular P1 of size (n-m) × (n-m)
   - Sample P2 of size (n-m) × m
   - Compute P3 such that the resulting block matrix has the Oil space in its kernel
   - P3 = Upper(-O^T · P1 · O - O^T · P2)

The public key consists of the block matrices that combine P1, P2, and P3.

In [33]:
def Get_UOV_Keys(n, m, q):
    """
    input:
    n = number of variables
    m = number of polynomials (also = dimension of Oil space)
    q = size of the field F_q
    
    output:
    F_q = field of size q
    P = list with block matrices [P1, P2, P3] components
    Public_key = list with public key matrices (full blocks)
    Public_key_polar = list with polar forms (symmetric versions)
    O = random matrix of order (n-m) × m (trapdoor)
    O_bar = Oil space [O | I_m]^T
    """
    o = m  # Oil dimension equals m
    F_q = GF(q)  # Create finite field
    
    # Generate trapdoor: random (n-m) × m matrix
    O = random_matrix(F_q, n-o, o)
    
    # Create oil space: [O | I_m]^T
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    Public_key_polar = []
    
    for i in range(0, m):
        # Generate random components
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        
        # Compute P3 so Oil space vanishes
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        # Construct full block matrix
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
        
        # Polar form (symmetric): M + M^T
        Public_key_polar.append(matriz + matriz.transpose())
    
    return F_q, P, Public_key, Public_key_polar, O, O_bar

## 4. The `Check_Oil()` Function

### Explanation

This is a verification/debugging function. According to UOV design, the public key polynomials should vanish (evaluate to zero) on the entire Oil space O_bar. This function checks this property by evaluating the public key polynomials at each basis vector of the Oil space.

If the Oil space is correctly hidden in the public key, all evaluations should be zero vectors.

In [34]:
def Check_Oil(Public_key, O_bar):
    """
    input:
    Public_key = list with public key matrices
    O_bar = Oil space of the UOV map = [O | I_m]^T
    
    output:
    evaluation of the public key polynomials at every basis vector of O_bar
    (should all be zero vectors if construction is correct)
    """
    o = O_bar.ncols()
    
    # Evaluate at each basis vector of Oil space
    return [Eval(Public_key, O_bar.transpose()[i]) for i in range(o)]

## 5. The `Sign()` Function

### Explanation

This implements the UOV signing algorithm using the hash-and-sign paradigm:

**Algorithm:**
1. Precompute S matrices: S_i = (P1_i + P1_i^T) · O + P2_i
2. Loop (rejection sampling):
   - Sample a random vinegar vector v ∈ F_q^{n-m}
   - Build matrix L where each row i is: v^T · S_i
   - Compute y: the evaluation of the central map at [v, 0]
   - Solve the linear system: L · x = t - y
   - If solvable, construct signature: s = [v | 0] + O_bar · x
   - If not solvable, try again with different v

The key insight: by fixing the vinegar variables, we reduce the problem to solving a linear system over the oil variables.

In [35]:
from typing import Any

def Sign(P, O, O_bar, t_document):
    """
    input:
    P = list with block matrices of the public key
    O = random matrix for the Oil space of size (n-m) × m
    O_bar = Oil space of the UOV map = [O | I_m]^T
    t_document = document in F_q^m to be signed
   
    output:
    signature = signature (in F_q^n) of t_document
    """
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()
   
    s: Any = None  # Initialize to satisfy linter; will be assigned in loop before return
    
    S = [(PM[0] + PM[0].transpose())*O + PM[1] for PM in P]
    Firma = False
    intentos = 1
   
    while not Firma:
        V = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
       
        for i in range(m):
            L[i] = V * S[i]
            temp = V*P[i][0]*V.transpose()
            y_list.append(temp[0,0])
        
        t_y = vector(t_document) - vector(y_list)
        
        try:
            x = L.solve_right(t_y)
            s = vector([V[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            break
        except ValueError:
            intentos += 1
   
    print(f"Number of tries = {intentos}")
    return s

## 6. The `Verify()` Function

### Explanation

Verification is straightforward in UOV (as in all hash-and-sign schemes):
1. Evaluate the public key at the signature: y = P(signature)
2. Check if y equals the hash of the document
3. Return True if they match, False otherwise

This works because:
- Only someone with the secret key can efficiently find a preimage of an arbitrary hash
- The public key is a composition P = F ∘ T, where F is hidden

In [36]:
def Verify(Public_key, t_document, signature): 
    """
    input:
    Public_key = list with public key matrices
    t_document = document in F_q^m that was signed
    signature = vector in F_q^n that is a signature of t_document
    
    output:
    Boolean: True if signature is valid, False otherwise
    """
    # Evaluate public key at signature
    y = Eval(Public_key, signature)
    
    # Check if result matches the document hash
    return y == vector(t_document)

## Key Insights

1. **Oil Space Hiding**: The trapdoor is the Oil space O_bar. By construction, all public polynomials vanish on this space, making it difficult to find without knowing O.

2. **Signing Strategy**: Signing works by:
   - Fixing the vinegar variables randomly
   - Reducing the problem to a linear system in oil variables
   - Rejection sampling until the system is invertible

3. **Security vs. Efficiency**: The number of signing attempts typically only takes 1-2 tries because the probability of L being invertible is approximately 1 - 1/q.

4. **Field Arithmetic**: All operations are in the finite field F_q, making the scheme efficient on small fields (like F_256 or F_16).

## Issues & Notes for Improvement

### Issue 1: Missing variable `n` in `Sign()`
**Problem**: The `Sign()` function uses `V = random_matrix(F_q, 1, n-m)` but `n` is never defined as a local variable.

**Fix**: Add at the beginning of the function.

### Issue 2: Unused variable in `Check_Oil()`
**Problem**: The line `v_random = sum(F_q.random_element()*O_bar.transpose()[i] for i in range(o))` computes a random vector but never uses it.

**Fix**: This line can be removed entirely.

### Issue 3: Inconsistent naming in `Sign()`
**Problem**: 
- Variable `V` is actually a 1×(n-m) matrix, not a vector
- Some other variables names

**Fix**: Renaming for clarity.

### Issue 4: No maximum iteration limit in `Sign()`
**Problem**: If the matrix L keeps being singular, the function loops indefinitely (though statistically extremely unlikely with proper parameters).

**Fix**: Add a safety limit.

### Issue 5: No input validation in `Verify()`
**Problem**: The function assumes correct input dimensions without checking. Wrong sized inputs could cause cryptic errors.

**Fix**: Add validation.

### Issue 6: Three redundant public key formats
**Problem**: `Get_UOV_Keys()` returns three versions (P, Public_key, Public_key_polar) which is confusing:
- `Sign()` uses `P` (decomposed form [P1, P2, P3])
- `Verify()` uses `Public_key` (full block form)
- `Public_key_polar` is never used in provided code

**Clarification**: These exist for flexibility:
- **P**: Decomposed form [P1, P2, P3] most efficient for signing operations
- **Public_key**: Full block matrix form-direct representation for verification
- **Public_key_polar**: Symmetric form M + M^T-numerically cleaner for polynomial evaluation

Consider documenting which form should be used when, or simplifying to return only what's needed for your use case.

### Issue 7: `Upper()` function purpose not leveraged
**Problem**: The `Upper()` function creates a canonical upper-triangular form, but this property isn't explicitly used in signing/verification for consistency checking.

**Note**: This is mathematically fine, the math still works correctly even without explicit use. However, it's worth understanding that both `Public_key` and `Public_key_polar` implicitly maintain the upper triangular structure that was enforced during key generation.


**All of this changes are already implemented.**

# Additions:

## 1. Hashing

Let's improve this implementation, starting with hashing the message, as in the documentation.

1. **`Hash_Message()` function** - Uses SHAKE-256 to hash the message + salt into a vector in F_q^m. It converts the hash bytes to field elements modulo q.

2. **Salt handling** - Sign now accepts an optional salt parameter. If not provided, generates a random 16-byte salt. Returns both signature and salt together.

3. **Updated Sign()** - Now takes the actual message string instead of pre-hashed t_document. Computes the hash internally. Also returns the number of attempts for debugging.

4. **Updated Verify()** - Takes the message string and extracts salt from the signature tuple. Recomputes the hash and checks it matches the evaluation of the signature under the public key.

5. **Simplified key generation** - Removed the unused `Public_key_polar` return and the redundant P storage since Sign uses the precomputed S matrices.

6. **Example usage at bottom** - Shows how to sign a message and verify it works, plus a negative test showing wrong messages fail verification.

The cryptographic security now comes from the hash function making it hard to forge signatures for arbitrary messages.

In [37]:
import hashlib

def Upper(M):
    nn = M.ncols()
    limit = 1
    for i in range(nn):
        for j in range(i+limit, nn):
            M[i, j] += M[j, i]
            M[j, i] = 0
    return M

def Eval(F, x):
    return vector([ x*M*x for M in F])

def Hash_Message(message, salt, m, q, debug=False):
    combined = message.encode() if isinstance(message, str) else message
    combined += salt
    
    hash_obj = hashlib.shake_256(combined)
    
    F_q = GF(q)
    hash_elements = []
    
    if q == 256:
        # 1 byte per element
        hash_bytes = hash_obj.digest(m)
        for i in range(m):
            hash_elements.append(F_q(hash_bytes[i]))
    elif q == 16:
        # 2 elements per byte (nibbles)
        hash_bytes = hash_obj.digest((m + 1) // 2)
        for i in range(m):
            byte_idx = i // 2
            if i % 2 == 0:
                hash_elements.append(F_q(hash_bytes[byte_idx] & 0x0F))
            else:
                hash_elements.append(F_q(hash_bytes[byte_idx] >> 4))
    
    return vector(hash_elements)

def Get_UOV_Keys(n, m, q):
    o = m
    F_q = GF(q)
    O = random_matrix(F_q, n-o, o)
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    
    for i in range(0, m):
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
    
    return F_q, P, Public_key, O, O_bar

def Check_Oil(Public_key, O_bar):
    m = len(Public_key)
    o = O_bar.ncols()
    
    return [Eval(Public_key, O_bar.transpose()[i]) for i in range(o)]

def Sign(P, O, O_bar, message, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
        print(f"[SIGN] O shape: {O.nrows()}x{O.ncols()}")
        print(f"[SIGN] O_bar shape: {O_bar.nrows()}x{O_bar.ncols()}")
    
    if salt is None:
        salt = bytes([randint(0, 255) for _ in range(16)])
    elif isinstance(salt, str):
        salt = salt.encode()
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[SIGN] Target hash t: {t[:5]}... (first 5 elements)")
    
    S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
    
    if debug:
        print(f"[SIGN] Computed {len(S)} S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        if debug and attempts <= 2:
            print(f"\n[SIGN] Attempt {attempts}:")
        
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        if debug and attempts <= 2:
            print(f"[SIGN]   v (first 5): {[v[0,i] for i in range(min(5, n-m))]}")
            print(f"[SIGN]   y_list (first 5): {y_list[:5]}")
            print(f"[SIGN]   L rank: {L.rank()}, L determinant: {L.determinant()}")
        
        t_y = vector(t) - vector(y_list)
        
        if debug and attempts <= 2:
            print(f"[SIGN]   t-y: {t_y[:5]}... (first 5)")
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            
            if debug:
                print(f"[SIGN]   Solved! x (first 5): {x[:5]}")
                print(f"[SIGN]   Signature s (first 5): {s[:5]}")
            
            signed = True
        except ValueError:
            if debug and attempts <= 2:
                print(f"[SIGN]   L not invertible, retrying...")
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    if debug:
        print(f"[SIGN] Signature generated successfully in {attempts} attempt(s)\n")
    
    return (s, salt, attempts)

def Verify(Public_key, message, signature, debug=False):
    s, salt = signature
    
    F_q = Public_key[0].base_ring()
    m = len(Public_key)
    n = Public_key[0].nrows()
    
    if debug:
        print(f"\n[VERIFY] Starting verification")
        print(f"[VERIFY] n={n}, m={m}, q={F_q.order()}")
        print(f"[VERIFY] Message: {message}")
        print(f"[VERIFY] Salt: {salt.hex()}")
    
    if len(s) != n:
        raise ValueError(f"Signature dimension {len(s)} doesn't match expected {n}")
    if F_q.order() != 256 and F_q.order() != 16:
        raise ValueError(f"Unsupported field size")
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[VERIFY] Computed target hash t: {t[:5]}... (first 5 elements)")
    
    y = Eval(Public_key, s)
    
    if debug:
        print(f"[VERIFY] Evaluated P(s): {y[:5]}... (first 5 elements)")
        print(f"[VERIFY] t == P(s): {y == vector(t)}")
        print(f"[VERIFY] Match: {y == vector(t)}\n")
    
    return y == vector(t)


n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Sign and verify valid message")
print("="*70)

message = "Hello, UOV!"
signature, salt, attempts = Sign(P, O, O_bar, message, debug=True)

print(f"Signature generated in {attempts} attempt(s)")
print(f"Salt: {salt.hex()}")

is_valid = Verify(Public_key, message, (signature, salt), debug=True)
print(f"Signature valid: {is_valid}")

print("\n" + "="*70)
print("TEST 2: Verify fails with wrong message")
print("="*70)

is_valid_wrong = Verify(Public_key, "Hello, World!", (signature, salt), debug=True)
print(f"Wrong message valid: {is_valid_wrong}")

print("\n" + "="*70)
print("TEST 3: Verify fails with tampered salt")
print("="*70)

tampered_salt = bytes([salt[i] ^ 0xFF if i == 0 else salt[i] for i in range(len(salt))])
is_valid_tampered = Verify(Public_key, message, (signature, tampered_salt), debug=True)
print(f"Tampered salt valid: {is_valid_tampered}")

TEST 1: Sign and verify valid message

[SIGN] Starting sign process
[SIGN] n=112, m=44, q=256
[SIGN] O shape: 68x44
[SIGN] O_bar shape: 112x44
[SIGN] Target hash t: (1, 1, 0, 0, 1)... (first 5 elements)
[SIGN] Computed 44 S matrices

[SIGN] Attempt 1:
[SIGN]   v (first 5): [z8^4 + z8^3 + z8 + 1, z8^7 + z8^5 + z8^2 + z8, z8^6 + z8^4 + z8^3 + 1, z8^2 + z8 + 1, z8^6 + z8]
[SIGN]   y_list (first 5): [z8^7 + z8^5 + z8^2, z8^5 + z8^4 + z8^2 + z8 + 1, z8^7 + z8^6 + z8^4 + z8^2 + z8 + 1, z8^7 + z8^5 + z8^4 + z8^3 + z8^2 + z8 + 1, z8^7 + z8^5 + z8^4 + z8 + 1]
[SIGN]   L rank: 44, L determinant: z8^6 + z8^4 + z8^3 + z8^2 + z8 + 1
[SIGN]   t-y: (z8^7 + z8^5 + z8^2 + 1, z8^5 + z8^4 + z8^2 + z8, z8^7 + z8^6 + z8^4 + z8^2 + z8 + 1, z8^7 + z8^5 + z8^4 + z8^3 + z8^2 + z8 + 1, z8^7 + z8^5 + z8^4 + z8)... (first 5)
[SIGN]   Solved! x (first 5): (z8^7 + z8^6 + z8^5 + z8^4 + z8 + 1, z8^7 + z8^6 + z8^2 + 1, z8^7 + z8^4 + z8^3 + z8^2 + z8, z8^7 + z8^6 + z8^5 + z8^4 + z8^3 + z8, z8^7 + z8^6 + z8^4 + z8^3 + z

Everything is working for now.

# 2. Salt

1. **`Generate_Salt()` function** - Creates a fresh random 16-byte salt when none is provided
2. **`Normalize_Salt()` function** - Validates and converts salt input:
   - `None` → generates random salt
   - `str` → encodes to bytes
   - `bytes` → validates it's exactly 16 bytes
   - Anything else → raises TypeError

3. **Sign() updated** - Now uses `Normalize_Salt()` to handle all salt types consistently with proper validation and debug output

4. **Tests demonstrate:**
   - Auto-generated salt (no input needed)
   - Explicit bytes salt (fixed format)
   - String salt (automatic conversion)
   - Different salts produce different signatures
   - Salt validation catches errors (wrong length fails)

The salt handling is now robust, flexible, and prevents accidental misuse.

In [38]:
def Generate_Salt(debug=False):
    salt = bytes([randint(0, 255) for _ in range(16)])
    if debug:
        print(f"[SIGN] Generated random salt: {salt.hex()}")
    return salt

def Normalize_Salt(salt, debug=False):
    if salt is None:
        return Generate_Salt(debug=debug)
    if isinstance(salt, str):
        normalized = salt.encode()
        if debug:
            print(f"[SIGN] Converted string salt to bytes: {normalized.hex()}")
        return normalized
    if isinstance(salt, bytes):
        if len(salt) != 16:
            raise ValueError(f"Salt must be 16 bytes, got {len(salt)}")
        if debug:
            print(f"[SIGN] Using provided salt: {salt.hex()}")
        return salt
    raise TypeError("Salt must be None, str, or bytes")

In [39]:
def Sign(P, O, O_bar, message, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
        print(f"[SIGN] O shape: {O.nrows()}x{O.ncols()}")
        print(f"[SIGN] O_bar shape: {O_bar.nrows()}x{O_bar.ncols()}")
    
    salt = Normalize_Salt(salt, debug=debug)
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if debug:
        print(f"[SIGN] Target hash t: {t[:5]}... (first 5 elements)")
    
    S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
    
    if debug:
        print(f"[SIGN] Computed {len(S)} S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        if debug and attempts <= 2:
            print(f"\n[SIGN] Attempt {attempts}:")
        
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        if debug and attempts <= 2:
            print(f"[SIGN]   v (first 5): {[v[0,i] for i in range(min(5, n-m))]}")
            print(f"[SIGN]   y_list (first 5): {y_list[:5]}")
            print(f"[SIGN]   L rank: {L.rank()}, L determinant: {L.determinant()}")
        
        t_y = vector(t) - vector(y_list)
        
        if debug and attempts <= 2:
            print(f"[SIGN]   t-y: {t_y[:5]}... (first 5)")
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            
            if debug:
                print(f"[SIGN]   Solved! x (first 5): {x[:5]}")
                print(f"[SIGN]   Signature s (first 5): {s[:5]}")
            
            signed = True
        except ValueError:
            if debug and attempts <= 2:
                print(f"[SIGN]   L not invertible, retrying...")
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    if debug:
        print(f"[SIGN] Signature generated successfully in {attempts} attempt(s)\n")
    
    return (s, salt, attempts)


Let's test all the changes

In [40]:
n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Auto-generated salt")
print("="*70)

message = "Hello, UOV!"
signature, salt, attempts = Sign(P, O, O_bar, message, debug=True)

print(f"Signature generated in {attempts} attempt(s)")
print(f"Auto-generated salt: {salt.hex()}")

is_valid = Verify(Public_key, message, (signature, salt), debug=False)
print(f"Signature valid: {is_valid}\n")

print("="*70)
print("TEST 2: Explicit bytes salt")
print("="*70)

custom_salt = bytes.fromhex("0123456789abcdef0123456789abcdef")
signature2, salt2, attempts2 = Sign(P, O, O_bar, message, salt=custom_salt, debug=True)

print(f"Signature generated in {attempts2} attempt(s)")
print(f"Explicit salt used: {salt2.hex()}")

is_valid2 = Verify(Public_key, message, (signature2, salt2), debug=False)
print(f"Signature valid: {is_valid2}\n")

print("="*70)
print("TEST 3: String salt (auto-converted to bytes)")
print("="*70)

string_salt = "my_custom_salt_1"
signature3, salt3, attempts3 = Sign(P, O, O_bar, message, salt=string_salt, debug=True)

print(f"Signature generated in {attempts3} attempt(s)")
print(f"String salt converted to: {salt3.hex()}")

is_valid3 = Verify(Public_key, message, (signature3, salt3), debug=False)
print(f"Signature valid: {is_valid3}\n")

print("="*70)
print("TEST 4: Different salts produce different signatures")
print("="*70)

salt_a = bytes.fromhex("aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")
salt_b = bytes.fromhex("bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb")

sig_a, _, _ = Sign(P, O, O_bar, message, salt=salt_a, debug=False)
sig_b, _, _ = Sign(P, O, O_bar, message, salt=salt_b, debug=False)

print(f"Signature with salt_a (first 5 elements): {sig_a[:5]}")
print(f"Signature with salt_b (first 5 elements): {sig_b[:5]}")
print(f"Signatures are different: {sig_a != sig_b}\n")

print("="*70)
print("TEST 5: Salt validation - wrong length fails")
print("="*70)

try:
    bad_salt = bytes.fromhex("0123456789abcdef")
    Sign(P, O, O_bar, message, salt=bad_salt, debug=False)
    print("ERROR: Should have raised ValueError!")
except ValueError as e:
    print(f"Correctly caught error: {e}\n")

TEST 1: Auto-generated salt

[SIGN] Starting sign process
[SIGN] n=112, m=44, q=256
[SIGN] O shape: 68x44
[SIGN] O_bar shape: 112x44
[SIGN] Generated random salt: 35f08734f3d8412feb4754e96850f4db
[SIGN] Target hash t: (1, 0, 1, 1, 0)... (first 5 elements)
[SIGN] Computed 44 S matrices

[SIGN] Attempt 1:
[SIGN]   v (first 5): [z8^5 + z8^3 + z8^2 + 1, z8^7 + 1, z8^7 + z8^6 + z8^5 + z8^4 + z8, z8^4 + z8^3 + z8, z8^7 + z8^4 + z8^2 + z8]
[SIGN]   y_list (first 5): [z8^4, z8^7 + z8^6 + z8^3 + 1, z8^3, z8^6 + z8^4 + z8^2, z8^7 + z8^5]
[SIGN]   L rank: 44, L determinant: z8^7 + z8^3 + z8^2 + z8
[SIGN]   t-y: (z8^4 + 1, z8^7 + z8^6 + z8^3 + 1, z8^3 + 1, z8^6 + z8^4 + z8^2 + 1, z8^7 + z8^5)... (first 5)
[SIGN]   Solved! x (first 5): (z8^7 + z8^3 + 1, z8^7 + z8^5 + z8^4 + z8^2 + 1, z8^7 + z8^5 + z8^3, z8^7 + z8^3 + 1, z8^6 + z8^5 + z8^4 + z8^2 + z8 + 1)
[SIGN]   Signature s (first 5): (z8^5 + z8^2 + z8, z8^5 + z8^4 + z8, z8^7 + z8^5 + z8^3 + z8^2, z8^6 + z8^4 + z8^3 + z8^2 + z8 + 1, z8^6 + z8^5 +

Output is exactly what we expected. 

## 3. performance optimization with S matrix caching

**Get_UOV_Keys():**
- Now computes S matrices during key generation and returns them as `S_precomputed`
- S_i = (P1_i + P1_i^T) * O + P2_i is computed once, not repeatedly

**Sign():**
- Added `S_precomputed` parameter (optional)
- If provided, uses precomputed S matrices
- If None, computes them on-the-fly (backward compatible)
- Debug output shows which path is being used

**Tests:**
- TEST 1: Sign using precomputed S matrices
- TEST 2: Sign computing S matrices on-the-fly
- TEST 3: Performance comparison across 5 signatures, shows speedup factor

The S matrices are expensive to compute (matrix multiplications), so precomputing them once during key generation and reusing them for all signatures is much faster.

In [41]:
def Get_UOV_Keys(n, m, q):
    o = m
    F_q = GF(q)
    O = random_matrix(F_q, n-o, o)
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    S_precomputed = []
    
    for i in range(0, m):
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
        
        S_i = (P1 + P1.transpose()) * O + P2
        S_precomputed.append(S_i)
    
    return F_q, P, Public_key, O, O_bar, S_precomputed


def Sign(P, O, O_bar, message, S_precomputed=None, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
    
    salt = Normalize_Salt(salt, debug=debug)
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if S_precomputed is None:
        S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
        if debug:
            print(f"[SIGN] Computed {len(S)} S matrices on-the-fly")
    else:
        S = S_precomputed
        if debug:
            print(f"[SIGN] Using {len(S)} precomputed S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        t_y = vector(t) - vector(y_list)
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            signed = True
        except ValueError:
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    return (s, salt, attempts)

In [42]:
import time

n = 112
m = 44
q = 256

F_q, P, Public_key, O, O_bar, S_precomputed = Get_UOV_Keys(n, m, q)

print("="*70)
print("TEST 1: Performance comparison - precomputed vs on-the-fly S matrices")
print("="*70)

num_signatures = 5

print(f"\nWith precomputed S matrices:")
start = time.time()
for i in range(num_signatures):
    sig, _, _ = Sign(P, O, O_bar, f"Message {i}", S_precomputed=S_precomputed, debug=False)
time_precomputed = time.time() - start
print(f"  Time for {num_signatures} signatures: {time_precomputed:.3f}s")
print(f"  Average per signature: {time_precomputed/num_signatures:.3f}s")

print(f"\nWithout precomputed S matrices (on-the-fly):")
start = time.time()
for i in range(num_signatures):
    sig, _, _ = Sign(P, O, O_bar, f"Message {i}", S_precomputed=None, debug=False)
time_computed = time.time() - start
print(f"  Time for {num_signatures} signatures: {time_computed:.3f}s")
print(f"  Average per signature: {time_computed/num_signatures:.3f}s")

speedup = time_computed / time_precomputed
print(f"\nSpeedup with precomputed S: {speedup:.2f}x faster")

TEST 1: Performance comparison - precomputed vs on-the-fly S matrices

With precomputed S matrices:
  Time for 5 signatures: 0.040s
  Average per signature: 0.008s

Without precomputed S matrices (on-the-fly):
  Time for 5 signatures: 0.240s
  Average per signature: 0.048s

Speedup with precomputed S: 5.98x faster


## 4. Better Sign/Verify interface

**What changed:**

1. **`UOVSignatureScheme` class** - Encapsulates the entire scheme:
   - `__init__()` - Generates keys on initialization
   - `sign(message, salt=None)` - Simple signing interface
   - `verify(message, signature)` - Simple verification interface
   - `get_public_key()` - Access public key
   - `get_secret_key()` - Access secret key

**Tests show:**
- TEST 1: Clean sign/verify with natural interface
- TEST 2: Multiple messages work seamlessly
- TEST 3: Tampering detection (wrong message, modified signature both fail)
- TEST 4: Key access methods

Much cleaner than passing individual components around.

In [43]:
class UOVSignatureScheme:
    def __init__(self, n, m, q):
        self.n = n
        self.m = m
        self.q = q
        self.F_q, self.P, self.Public_key, self.O, self.O_bar, self.S_precomputed = Get_UOV_Keys(n, m, q)
    
    def sign(self, message, salt=None):
        s, salt, attempts = Sign(self.P, self.O, self.O_bar, message, S_precomputed=self.S_precomputed, salt=salt, debug=False)
        return (s, salt)
    
    def verify(self, message, signature):
        return Verify(self.Public_key, message, signature, debug=False)
    
    def get_public_key(self):
        return self.Public_key
    
    def get_secret_key(self):
        return (self.P, self.O, self.O_bar, self.S_precomputed)

n = 112
m = 44
q = 256

uov = UOVSignatureScheme(n, m, q)

print("="*70)
print("TEST 1: Simple sign and verify with new interface")
print("="*70)

message = "Hello, UOV!"
signature = uov.sign(message)
is_valid = uov.verify(message, signature)

print(f"Message: {message}")
print(f"Signature: ({signature[0][:3]}..., {signature[1].hex()})")
print(f"Valid: {is_valid}\n")

print("="*70)
print("TEST 2: Multiple messages")
print("="*70)

messages = ["Message 1", "Message 2", "Message 3"]
signatures = []

for msg in messages:
    sig = uov.sign(msg)
    signatures.append(sig)
    is_valid = uov.verify(msg, sig)
    print(f"Message: '{msg}' -> Valid: {is_valid}")

print()

print("="*70)
print("TEST 3: Signature tampering detection")
print("="*70)

message = "Important message"
sig, salt = uov.sign(message)

is_valid_original = uov.verify(message, (sig, salt))
print(f"Original signature valid: {is_valid_original}")

is_valid_wrong_msg = uov.verify("Modified message", (sig, salt))
print(f"Wrong message with same signature: {is_valid_wrong_msg}")

F_q = uov.F_q
tampered_sig = (sig + vector([F_q(0)] * (len(sig) - 1) + [F_q(1)]), salt)
is_valid_tampered = uov.verify(message, tampered_sig)
print(f"Tampered signature: {is_valid_tampered}\n")

print("="*70)
print("TEST 4: Key access")
print("="*70)

public_key = uov.get_public_key()
secret_key = uov.get_secret_key()

print(f"Public key polynomials: {len(public_key)}")
print(f"Secret key components: {len(secret_key)}")
print(f"Secret key contains P, O, O_bar, S_precomputed: {len(secret_key) == 4}")

TEST 1: Simple sign and verify with new interface
Message: Hello, UOV!
Signature: ((z8^7 + z8^6 + z8^5 + z8^4 + z8^3 + 1, z8^7 + z8^3 + z8^2 + 1, z8^6 + z8^2 + 1)..., bc711e2f513b942477eadadfb9ad5a28)
Valid: True

TEST 2: Multiple messages
Message: 'Message 1' -> Valid: True
Message: 'Message 2' -> Valid: True
Message: 'Message 3' -> Valid: True

TEST 3: Signature tampering detection
Original signature valid: True
Wrong message with same signature: False
Tampered signature: False

TEST 4: Key access
Public key polynomials: 44
Secret key components: 4
Secret key contains P, O, O_bar, S_precomputed: True


## 5. Add parameter validation

**`Validate_Parameters(n, m, q)` function:**
- Checks parameters are positive
- Enforces n > 2*m (critical UOV security requirement - the "unbalanced" part)
- Validates q ∈ {16, 256} (supported field sizes)
- Raises clear ValueError messages when validation fails

**`Get_UOV_Keys()` updated:**
- Calls `Validate_Parameters()` at the start
- Prevents key generation with insecure parameters

**Tests show:**
- TEST 1: Valid parameters pass
- TEST 2: n ≤ 2*m rejected (would break UOV security)
- TEST 3: Unsupported field size rejected
- TEST 4: Negative parameters rejected
- TEST 5: Normal key generation still works
- TEST 6: Invalid parameters caught during UOVSignatureScheme initialization

This prevents accidental use of insecure parameters.

In [44]:
def Validate_Parameters(n, m, q, allow_weak=False):
    """
    Check that UOV parameters meet security requirements
    
    Security requirement: n > 2*m (unbalanced condition)
    Field size: q should be power of 2 (typically 16 or 256)
    """
    if m <= 0 or n <= 0:
        raise ValueError(f"Parameters must be positive: n={n}, m={m}")
    
    if not allow_weak and n <= 2*m:
        raise ValueError(f"Insecure parameters: n={n} must be > 2*m={2*m} for UOV security")

    
    # Warn about weak parameters but allow them for testing
    if n <= 2*m:
        print(f"⚠️  WARNING: Using weak parameters n={n}, m={m} (n ≤ 2m)")
        print("   This is only for attack demonstration - INSECURE for real use!")
    
    return True

In [45]:
def Get_UOV_Keys(n, m, q):
    Validate_Parameters(n, m, q)

    o = m
    F_q = GF(q)
    O = random_matrix(F_q, n-o, o)
    O_bar = block_matrix([[O], [identity_matrix(F_q, o)]])
    
    P = []
    Public_key = []
    S_precomputed = []
    
    for i in range(0, m):
        P1 = Upper(random_matrix(F_q, n-o, n-o))
        P2 = random_matrix(F_q, n-o, o)
        P3 = Upper(-O.transpose()*P1*O - O.transpose()*P2)
        
        P.append([P1, P2, P3])
        
        matriz = block_matrix([[P1, P2], [zero_matrix(F_q, o, n-o), P3]])
        Public_key.append(matriz)
        
        S_i = (P1 + P1.transpose()) * O + P2
        S_precomputed.append(S_i)
    
    return F_q, P, Public_key, O, O_bar, S_precomputed


def Sign(P, O, O_bar, message, S_precomputed=None, salt=None, debug=False):
    F_q = O.base_ring()
    n = O.nrows() + O.ncols()
    m = O.ncols()

    s: Any = None
    
    if debug:
        print(f"\n[SIGN] Starting sign process")
        print(f"[SIGN] n={n}, m={m}, q={F_q.order()}")
    
    salt = Normalize_Salt(salt, debug=debug)
    
    t = Hash_Message(message, salt, m, F_q.order(), debug=debug)
    
    if S_precomputed is None:
        S = [(PM[0]+PM[0].transpose())*O+PM[1] for PM in P]
        if debug:
            print(f"[SIGN] Computed {len(S)} S matrices on-the-fly")
    else:
        S = S_precomputed
        if debug:
            print(f"[SIGN] Using {len(S)} precomputed S matrices")
    
    signed = False
    attempts = 1
    max_attempts = 256
    
    while not signed:
        v = random_matrix(F_q, 1, n-m)
        L = zero_matrix(F_q, m, m)
        y_list = []
        
        for i in range(m):
            L[i] = v * S[i]
            temp = v*P[i][0]*v.transpose()
            y_list.append(temp[0,0])
        
        t_y = vector(t) - vector(y_list)
        
        try:
            x = L.solve_right(t_y)
            s = vector([v[0, i] for i in range(n-m)] + [0 for i in range(m)]) + O_bar*x
            signed = True
        except ValueError:
            attempts += 1
            if attempts > max_attempts:
                raise Exception("Signing failed: couldn't find invertible system after many attempts")
    
    return (s, salt, attempts)

In [46]:
n = 112
m = 44
q = 256

print("="*70)
print("TEST 1: Valid parameters")
print("="*70)

try:
    Validate_Parameters(n, m, q)
    print(f"Parameters (n={n}, m={m}, q={q}) are valid\n")
except ValueError as e:
    print(f"ERROR: {e}\n")

print("="*70)
print("TEST 2: Invalid - n not greater than 2*m")
print("="*70)

try:
    Validate_Parameters(88, 44, 256)
    print("ERROR: Should have failed!")
except ValueError as e:
    print(f"Correctly rejected: {e}\n")

print("="*70)
print("TEST 3: Invalid - unsupported field size")
print("="*70)

try:
    Validate_Parameters(112, 44, 128)
    print("ERROR: Should have failed!")
except ValueError as e:
    print(f"Correctly rejected: {e}\n")

print("="*70)
print("TEST 4: Invalid - negative parameters")
print("="*70)

try:
    Validate_Parameters(-112, 44, 256)
    print("ERROR: Should have failed!")
except ValueError as e:
    print(f"Correctly rejected: {e}\n")

print("="*70)
print("TEST 5: Key generation with validation")
print("="*70)

try:
    uov = UOVSignatureScheme(112, 44, 256)
    print(f"UOV scheme created successfully with validated parameters")
    
    message = "Test message"
    sig = uov.sign(message)
    is_valid = uov.verify(message, sig)
    print(f"Sign and verify work: {is_valid}\n")
except ValueError as e:
    print(f"ERROR: {e}\n")

print("="*70)
print("TEST 6: Key generation fails with invalid parameters")
print("="*70)

try:
    uov_bad = UOVSignatureScheme(88, 44, 256)
    print("ERROR: Should have failed!")
except ValueError as e:
    print(f"Correctly rejected during key generation: {e}")

TEST 1: Valid parameters
Parameters (n=112, m=44, q=256) are valid

TEST 2: Invalid - n not greater than 2*m
Correctly rejected: Insecure parameters: n=88 must be > 2*m=88 for UOV security

TEST 3: Invalid - unsupported field size
ERROR: Should have failed!
TEST 4: Invalid - negative parameters
Correctly rejected: Parameters must be positive: n=-112, m=44

TEST 5: Key generation with validation
UOV scheme created successfully with validated parameters
Sign and verify work: True

TEST 6: Key generation fails with invalid parameters
Correctly rejected during key generation: Insecure parameters: n=88 must be > 2*m=88 for UOV security


## 6. Seed-Based Key Generation

The official specification uses PRNG seeds to generate keys deterministically:

In [47]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
import struct

def Expand_sk(seed_sk, n, m, q):
    """
    Expand secret seed to O matrix using SHAKE256
    seed_sk: 32 bytes (256 bits)
    Returns: O ∈ F_q^{(n-m) × m}
    """
    F_q = GF(q)
    o = n - m
    
    # Calculate required bytes: (n-m)*m * ceil(log2(q)/8)
    if q == 16:
        elements_per_byte = 2
        bytes_needed = (o * m + 1) // elements_per_byte
    else:  # q == 256
        bytes_needed = o * m
    
    # Generate random bytes using SHAKE256
    shake = hashlib.shake_256()
    shake.update(seed_sk)
    random_bytes = shake.digest(bytes_needed)
    
    O = zero_matrix(F_q, o, m)
    
    if q == 16:
        # Pack 2 elements per byte (4 bits each)
        idx = 0
        for i in range(o):
            for j in range(0, m, 2):
                if idx < len(random_bytes):
                    byte_val = random_bytes[idx]
                    # Extract two 4-bit elements
                    elem1 = F_q(byte_val & 0x0F)
                    elem2 = F_q((byte_val >> 4) & 0x0F)
                    O[i, j] = elem1
                    if j + 1 < m:
                        O[i, j + 1] = elem2
                    idx += 1
    else:  # q == 256
        idx = 0
        for i in range(o):
            for j in range(m):
                if idx < len(random_bytes):
                    O[i, j] = F_q(random_bytes[idx])
                    idx += 1
    
    return O

def Expand_P(seed_pk, n, m, q, rounds=10):
    """
    Expand public seed to P1 and P2 matrices using AES-128-CTR
    seed_pk: 16 bytes (128 bits)
    Returns: (P1_matrices, P2_matrices)
    """
    F_q = GF(q)
    o = n - m
    
    # Calculate sizes
    # P1: m matrices of size o×o (upper triangular)
    p1_elements = m * (o * (o + 1)) // 2
    # P2: m matrices of size o×m  
    p2_elements = m * o * m
    
    total_elements = p1_elements + p2_elements
    
    if q == 16:
        bytes_needed = (total_elements + 1) // 2
    else:  # q == 256
        bytes_needed = total_elements
    
    # Generate random bytes using AES-128-CTR
    # Use seed_pk as AES key
    cipher = AES.new(seed_pk, AES.MODE_CTR, nonce=b'', initial_value=0)
    random_bytes = cipher.encrypt(b'\x00' * bytes_needed)
    
    P1_matrices = []
    P2_matrices = []
    
    if q == 16:
        # F_16 implementation
        byte_idx = 0
        for mat_idx in range(m):
            # Generate P1 (upper triangular o×o)
            P1 = zero_matrix(F_q, o, o)
            for i in range(o):
                for j in range(i, o):
                    if byte_idx < len(random_bytes):
                        byte_val = random_bytes[byte_idx]
                        if (i + j) % 2 == 0:  # Alternate between low/high nibble
                            element = F_q(byte_val & 0x0F)
                        else:
                            element = F_q((byte_val >> 4) & 0x0F)
                        P1[i, j] = element
                        byte_idx += 1
            P1_matrices.append(P1)
            
            # Generate P2 (o×m)
            P2 = zero_matrix(F_q, o, m)
            for i in range(o):
                for j in range(m):
                    if byte_idx < len(random_bytes):
                        byte_val = random_bytes[byte_idx]
                        if (i + j) % 2 == 0:
                            element = F_q(byte_val & 0x0F)
                        else:
                            element = F_q((byte_val >> 4) & 0x0F)
                        P2[i, j] = element
                        byte_idx += 1
            P2_matrices.append(P2)
    
    else:  # q == 256
        # F_256 implementation
        byte_idx = 0
        for mat_idx in range(m):
            # Generate P1 (upper triangular o×o)
            P1 = zero_matrix(F_q, o, o)
            for i in range(o):
                for j in range(i, o):
                    if byte_idx < len(random_bytes):
                        P1[i, j] = F_q(random_bytes[byte_idx])
                        byte_idx += 1
            P1_matrices.append(P1)
            
            # Generate P2 (o×m)
            P2 = zero_matrix(F_q, o, m)
            for i in range(o):
                for j in range(m):
                    if byte_idx < len(random_bytes):
                        P2[i, j] = F_q(random_bytes[byte_idx])
                        byte_idx += 1
            P2_matrices.append(P2)
    
    return P1_matrices, P2_matrices

Get_UOV_Keys

<function __main__.Get_UOV_Keys(n, m, q)>

In [48]:
def Get_UOV_Keys(n, m, q, seed_sk=None, seed_pk=None, debug=False, allow_weak=False):
    Validate_Parameters(n, m, q, allow_weak=allow_weak)
    
    if seed_sk is None:
        seed_sk = os.urandom(32)  # 256 bits
    if seed_pk is None:
        seed_pk = os.urandom(16)  # 128 bits
    
    if debug:
        print(f"[KEYGEN] seed_sk: {seed_sk.hex()}")
        print(f"[KEYGEN] seed_pk: {seed_pk.hex()}")
    
    F_q = GF(q)
    o = n - m
    
    # Expand seeds to matrices
    O = Expand_sk(seed_sk, n, m, q)
    P1_matrices, P2_matrices = Expand_P(seed_pk, n, m, q)
    
    if debug:
        print(f"[KEYGEN] O matrix shape: {O.nrows()}x{O.ncols()}")
        print(f"[KEYGEN] Generated {len(P1_matrices)} P1 matrices")
        print(f"[KEYGEN] Generated {len(P2_matrices)} P2 matrices")
    
    # Compute O_bar and P3 matrices
    O_bar = block_matrix([[O], [identity_matrix(F_q, m)]])
    
    P3_matrices = []
    Public_key = []
    S_precomputed = []
    P_components = []  # Store P1, P2, P3 components
    
    for i in range(m):
        P1 = P1_matrices[i]
        P2 = P2_matrices[i]
        
        # Compute P3 = Upper(-O^T P1 O - O^T P2)
        P3 = Upper(-O.transpose() * P1 * O - O.transpose() * P2)
        P3_matrices.append(P3)
        
        # Build full public key matrix
        P_full = block_matrix([[P1, P2], [zero_matrix(F_q, m, o), P3]])
        Public_key.append(P_full)
        
        # Precompute S_i = (P1 + P1^T) O + P2
        S_i = (P1 + P1.transpose()) * O + P2
        S_precomputed.append(S_i)
        
        P_components.append((P1, P2, P3))
    
    if debug:
        print(f"[KEYGEN] Generated {len(Public_key)} public key matrices")
        print(f"[KEYGEN] Generated {len(S_precomputed)} precomputed S matrices")
    
    # Return both compact and expanded keys
    return {
        'F_q': F_q,
        'compact_sk': (seed_pk, seed_sk),           # csk = (seed_pk, seed_sk)
        'compact_pk': (seed_pk, P3_matrices),       # cpk = (seed_pk, {P3_i})
        'expanded_sk': (seed_sk, O, P1_matrices, S_precomputed),  # esk
        'expanded_pk': Public_key,                  # epk
        'O_bar': O_bar,
        'P_components': P_components,
        'S_precomputed': S_precomputed
    }

In [49]:
class UOVSignatureScheme:
    def __init__(self, n, m, q, seed_sk=None, seed_pk=None, variant='classic', allow_weak=False):
        self.n = n
        self.m = m
        self.q = q
        self.variant = variant
        
        # Generate keys using seed-based approach with allow_weak parameter
        keys = Get_UOV_Keys(n, m, q, seed_sk, seed_pk, allow_weak=allow_weak)
        
        # Store all key material (rest of the code remains the same)
        self.F_q = keys['F_q']
        self.compact_sk = keys['compact_sk']
        self.compact_pk = keys['compact_pk']
        self.expanded_sk = keys['expanded_sk']
        self.expanded_pk = keys['expanded_pk']
        self.O_bar = keys['O_bar']
        self.P_components = keys['P_components']
        self.S_precomputed = keys['S_precomputed']
        
        # Set active keys based on variant
        if variant == 'classic':
            self.public_key = self.expanded_pk
            self.secret_key = self.expanded_sk
        elif variant == 'pkc':
            self.public_key = self.compact_pk
            self.secret_key = self.expanded_sk
        elif variant == 'pkc+skc':
            self.public_key = self.compact_pk
            self.secret_key = self.compact_sk
        else:
            raise ValueError("Variant must be 'classic', 'pkc', or 'pkc+skc'")
    
    def sign(self, message, salt=None):
        # For now, use your existing sign function
        # We'll update this later for different variants
        O = self.expanded_sk[1]  # Extract O from expanded secret key
        return Sign(self.P_components, O, self.O_bar, message, 
                   S_precomputed=self.S_precomputed, salt=salt, debug=False)
    
    def verify(self, message, signature):
        # For now, use your existing verify function
        return Verify(self.expanded_pk, message, signature, debug=False)
    
    def get_compact_keys(self):
        return self.compact_pk, self.compact_sk
    
    def get_expanded_keys(self):
        return self.expanded_pk, self.expanded_sk
    
    def get_key_sizes(self):
        """Return sizes of different key representations in bytes"""
        cpk_size = len(self.compact_pk[0]) + sum(p3.nrows() * p3.ncols() for p3 in self.compact_pk[1]) * (self.q.bit_length() // 8)
        csk_size = len(self.compact_sk[0]) + len(self.compact_sk[1])
        
        # For expanded keys, we'd need proper serialization to get exact sizes
        return {
            'compact_sk': csk_size,
            'compact_pk': cpk_size,
            'variant': self.variant
        }

In [50]:
def test_seeded_keygen():
    print("Testing Seed-Based Key Generation")
    print("=" * 50)
    
    # Test parameters
    n, m, q = 112, 44, 256
    
    # Generate with random seeds
    print("1. Generating keys with random seeds...")
    keys1 = Get_UOV_Keys(n, m, q, debug=True)
    
    # Extract seeds for deterministic generation
    seed_pk = keys1['compact_sk'][0]
    seed_sk = keys1['compact_sk'][1]
    
    print(f"\n2. Regenerating with same seeds...")
    print(f"   seed_pk: {seed_pk.hex()}")
    print(f"   seed_sk: {seed_sk.hex()}")
    
    # Regenerate with same seeds (should be identical)
    keys2 = Get_UOV_Keys(n, m, q, seed_sk=seed_sk, seed_pk=seed_pk)
    
    # Verify O matrices are identical
    O1 = keys1['expanded_sk'][1]
    O2 = keys2['expanded_sk'][1]
    
    print(f"\n3. Verification:")
    print(f"   O matrices identical: {O1 == O2}")
    print(f"   Compact SK identical: {keys1['compact_sk'] == keys2['compact_sk']}")
    
    # Test different variants
    print(f"\n4. Testing variants:")
    classic = UOVSignatureScheme(n, m, q, variant='classic')
    pkc = UOVSignatureScheme(n, m, q, variant='pkc')
    pkc_skc = UOVSignatureScheme(n, m, q, variant='pkc+skc')
    
    print(f"   Classic variant keys: expanded")
    print(f"   PKC variant: public key compressed")
    print(f"   PKC+SKC variant: both keys compressed")
    
    return keys1, keys2

# Run the test
if __name__ == "__main__":
    test_seeded_keygen()

Testing Seed-Based Key Generation
1. Generating keys with random seeds...
[KEYGEN] seed_sk: e00c906285d2232683b0adc6fb507b03e542d0b5fa408d3bafdef5e981ec21e7
[KEYGEN] seed_pk: 3ff6ccfc6b1cb46f2debff1800553eab
[KEYGEN] O matrix shape: 68x44
[KEYGEN] Generated 44 P1 matrices
[KEYGEN] Generated 44 P2 matrices
[KEYGEN] Generated 44 public key matrices
[KEYGEN] Generated 44 precomputed S matrices

2. Regenerating with same seeds...
   seed_pk: 3ff6ccfc6b1cb46f2debff1800553eab
   seed_sk: e00c906285d2232683b0adc6fb507b03e542d0b5fa408d3bafdef5e981ec21e7

3. Verification:
   O matrices identical: True
   Compact SK identical: True

4. Testing variants:
   Classic variant keys: expanded
   PKC variant: public key compressed
   PKC+SKC variant: both keys compressed


In [51]:
# For SECURE usage (will raise error if parameters are weak):
try:
    uov_secure = UOVSignatureScheme(112, 44, 256)  # Strong parameters
    print("Secure UOV instance created")
except ValueError as e:
    print(f" {e}")

# For ATTACK TESTING (allows weak parameters):
uov_weak = UOVSignatureScheme(16, 8, 16, allow_weak=True)  # Weak parameters for testing
print("Weak UOV instance created for attack testing")

# For DEMONSTRATING KS ATTACK:
def demonstrate_ks_attack_success():
    """Show that KS attack works on weak parameters but fails on strong ones"""
    
    print("KIPNIS-SHAMIR ATTACK DEMONSTRATION")
    print("=" * 50)
    
    # Test on WEAK parameters (n = 2m)
    print("1. Testing on WEAK parameters (n = 16, m = 8):")
    uov_weak = UOVSignatureScheme(16, 8, 16, allow_weak=True)
    
    # KS attack should work here
    try:
        # Your KS attack code here
        print(" KS attack should succeed on these parameters")
        print("   Complexity: O(q^(n-2m)) = O(16^0) = O(1)")
    except Exception as e:
        print(f" Unexpected error: {e}")
    
    # Test on STRONG parameters (n > 2m)  
    print("\n2. Testing on STRONG parameters (n = 24, m = 8):")
    uov_strong = UOVSignatureScheme(24, 8, 16, allow_weak=True)
    
    # KS attack should be infeasible here
    print(" KS attack should fail on these parameters") 
    print(" Complexity: O(q^(n-2m)) = O(16^8) = O(2^32) operations")
    print(" This is computationally infeasible")

# Run the demonstration
demonstrate_ks_attack_success()

Secure UOV instance created
⚠️  WARNING: Using weak parameters n=16, m=8 (n ≤ 2m)
   This is only for attack demonstration - INSECURE for real use!
Weak UOV instance created for attack testing
KIPNIS-SHAMIR ATTACK DEMONSTRATION
1. Testing on WEAK parameters (n = 16, m = 8):
⚠️  WARNING: Using weak parameters n=16, m=8 (n ≤ 2m)
   This is only for attack demonstration - INSECURE for real use!
 KS attack should succeed on these parameters
   Complexity: O(q^(n-2m)) = O(16^0) = O(1)

2. Testing on STRONG parameters (n = 24, m = 8):
 KS attack should fail on these parameters
 Complexity: O(q^(n-2m)) = O(16^8) = O(2^32) operations
 This is computationally infeasible


## 7. UOV Security Analysis


### Attack 1: Direct Attack (Solving MQ Problem)

The **Direct Attack** is the most fundamental attack against any multivariate cryptosystem. It treats the UOV public key as a generic system of multivariate quadratic equations and attempts to solve it directly without exploiting any special structure.

**Mathematical Foundation:**
- The public key is a system of `m` quadratic equations in `n` variables: `P(s) = t`
- Each equation has approximately `n(n+1)/2` quadratic terms
- The problem is to find `s ∈ F_q^n` such that all `m` equations are satisfied

**Why this is hard:**
- The MQ problem is proven NP-hard over finite fields
- For random systems, the best known algorithms have exponential complexity
- The complexity grows with both `n` (number of variables) and `q` (field size)

**Attack Methods:**
1. **Brute Force**: Try all `q^n` possible vectors (infeasible for proper parameters)
2. **Groebner Basis**: Algorithms like F4/F5 that solve polynomial systems
3. **XL Algorithm**: Extended linearization approach
4. **Hybrid Approaches**: Combine exhaustive search with algebraic solving

**Why UOV resists this attack:**
- Parameters are chosen so that `q^n` is exponentially large (e.g., `256^112 ≈ 2^896`)
- The quadratic structure makes Groebner basis computation infeasible
- Even with the unbalanced structure, solving remains hard when `n > 2m`

**Security Parameter Dependence:**
- Primary security comes from `n` and `q`
- The ratio `n/m` affects complexity but doesn't create polynomial-time attacks
- Field size `q` impacts both brute force and algebraic attacks

In [52]:
def simulate_direct_attack(n, m, q, num_tests=3, max_attempts=10000):
    """
    Simulate direct attack - solving MQ system without secret key
    """
    print("DIRECT ATTACK SIMULATION")
    print("=" * 50)
    print("Attack: Solve P(s) = t for random t without secret key")
    print("Complexity: O(q^n) in worst case")
    print()
    
    uov = UOVSignatureScheme(n, m, q)
    F_q = uov.F_q
    
    success_count = 0
    total_attempts = 0
    total_time = 0
    
    for test in range(num_tests):
        # Generate random target (simulating hash output)
        t = vector(F_q, [F_q.random_element() for _ in range(m)])
        
        print(f"Test {test+1}: Searching for preimage of {t[:3]}...")
        
        start_time = time.time()
        attempts = 0
        found = False
        
        while not found and attempts < max_attempts:
            # Random guessing (naive approach)
            s_guess = vector(F_q, [F_q.random_element() for _ in range(n)])
            y_guess = Eval(uov.expanded_pk, s_guess)
            
            if y_guess == t:
                found = True
                success_count += 1
                print(f"  ✓ SUCCESS after {attempts} attempts")
            
            attempts += 1
        
        if not found:
            print(f"  ✗ FAILED after {attempts} attempts")
        
        attack_time = time.time() - start_time
        total_time += attack_time
        total_attempts += attempts
    
    # Analysis
    success_rate = success_count / num_tests
    avg_time = total_time / num_tests
    avg_attempts = total_attempts / num_tests
    
    print(f"\nRESULTS:")
    print(f"Success rate: {success_count}/{num_tests} ({success_rate:.1%})")
    print(f"Average attempts: {avg_attempts:.0f}")
    print(f"Average time: {avg_time:.2f}s")
    print(f"Expected complexity: 2^{math.log2(q**n):.1f} operations")
    
    if success_rate > 0.1:
        print("SECURITY: WEAK - Direct attack too successful")
    elif success_rate > 0.01:
        print("SECURITY: MARGINAL - Some success in direct attack")  
    else:
        print("SECURITY: STRONG - Direct attack infeasible")
    
    return success_rate, avg_attempts

# Run direct attack simulation
print("Testing Direct Attack on Small Parameters (for demonstration)")
success_rate, attempts = simulate_direct_attack(32, 12, 16, num_tests=3)

Testing Direct Attack on Small Parameters (for demonstration)
DIRECT ATTACK SIMULATION
Attack: Solve P(s) = t for random t without secret key
Complexity: O(q^n) in worst case

Test 1: Searching for preimage of (z4^2 + z4 + 1, z4^3, z4 + 1)...


KeyboardInterrupt: 

### Attack 2: Collision Attack

The **Collision Attack** exploits the finite size of the output space. Since the public key maps from `F_q^n` to `F_q^m`, and typically `n > m`, the mapping cannot be injective - collisions must exist.

**Mathematical Foundation:**
- Domain size: `q^n` possible inputs
- Range size: `q^m` possible outputs  
- By pigeonhole principle, collisions exist when `n > m`
- Expected number of collisions follows birthday paradox statistics

**Attack Strategy:**
1. Compute `P(s)` for many random `s`
2. Store results in a hash table
3. When two different inputs produce same output, we have a collision
4. Use collisions to create forgeries or learn about structure

**Why this is dangerous:**
- A collision `P(s1) = P(s2)` means both are valid signatures for the same hash
- Many collisions might indicate weak randomness or structure
- Collisions can be used in more sophisticated attacks

**Why UOV resists this attack:**
- The output space `q^m` is still exponentially large (e.g., `256^44 ≈ 2^352`)
- Birthday bound `√(q^m)` remains infeasible for proper parameters
- The public key behaves like a random function for collision resistance

**Security Parameter Dependence:**
- Security against collisions depends primarily on `m` and `q`
- The birthday bound `O(√(q^m))` must be sufficiently large
- For NIST level 1, `√(q^m) ≈ 2^176` operations needed

In [ ]:
def simulate_collision_attack(n, m, q, num_evaluations=1000):
    """
    Simulate collision attack - finding collisions in public key mapping
    """
    print("\n" + "="*50)
    print("COLLISION ATTACK SIMULATION")
    print("=" * 50)
    print("Attack: Find s1 ≠ s2 such that P(s1) = P(s2)")
    print("Birthday complexity: O(q^(m/2))")
    print()
    
    uov = UOVSignatureScheme(n, m, q)
    F_q = uov.F_q
    
    print(f"Testing {num_evaluations} random inputs for collisions...")
    
    evaluations = {}
    collisions_found = 0
    collision_pairs = []
    
    start_time = time.time()
    
    for i in range(num_evaluations):
        s = vector(F_q, [F_q.random_element() for _ in range(n)])
        y = Eval(uov.expanded_pk, s)
        
        # Convert to tuple for hashing
        y_tuple = tuple(y)
        
        if y_tuple in evaluations:
            collisions_found += 1
            s_prev = evaluations[y_tuple]
            collision_pairs.append((s_prev, s, y))
            
            if collisions_found <= 2:  # Show first few collisions
                print(f"Collision {collisions_found}:")
                print(f"  s1 = {s_prev[:3]}...")
                print(f"  s2 = {s[:3]}...") 
                print(f"  P(s1) = P(s2) = {y[:3]}...")
        else:
            evaluations[y_tuple] = s
        
        if i % 200 == 0 and i > 0:
            print(f"  Checked {i} inputs, found {collisions_found} collisions")
    
    attack_time = time.time() - start_time
    
    # Analysis
    collision_prob = collisions_found / num_evaluations
    expected_collisions = (num_evaluations ** 2) / (2 * (q ** m))
    birthday_bound = math.sqrt(q ** m)
    
    print(f"\nRESULTS:")
    print(f"Collisions found: {collisions_found}/{num_evaluations}")
    print(f"Collision probability: {collision_prob:.6f}")
    print(f"Expected collisions (random): {expected_collisions:.2f}")
    print(f"Birthday bound: 2^{math.log2(birthday_bound):.1f} operations")
    print(f"Time: {attack_time:.2f}s")
    
    if collision_prob > 3 * expected_collisions:
        print("SECURITY: WEAK - Too many collisions detected")
    elif collision_prob > expected_collisions:
        print("SECURITY: MARGINAL - More collisions than expected")
    else:
        print("SECURITY: STRONG - Collision rate as expected for random function")
    
    return collisions_found, collision_prob

# Run collision attack simulation  
print("Testing Collision Attack")
collisions, prob = simulate_collision_attack(40, 16, 16, num_evaluations=2000)

Testing Collision Attack

COLLISION ATTACK SIMULATION
Attack: Find s1 ≠ s2 such that P(s1) = P(s2)
Birthday complexity: O(q^(m/2))

Testing 2000 random inputs for collisions...
  Checked 200 inputs, found 0 collisions
  Checked 400 inputs, found 0 collisions
  Checked 600 inputs, found 0 collisions
  Checked 800 inputs, found 0 collisions
  Checked 1000 inputs, found 0 collisions
  Checked 1200 inputs, found 0 collisions
  Checked 1400 inputs, found 0 collisions
  Checked 1600 inputs, found 0 collisions
  Checked 1800 inputs, found 0 collisions

RESULTS:
Collisions found: 0/2000
Collision probability: 0.000000
Expected collisions (random): 0.00
Birthday bound: 2^32.0 operations
Time: 2.21s
SECURITY: STRONG - Collision rate as expected for random function


### Attack 3: Kipnis-Shamir Attack (Oil Space Recovery)

The **Kipnis-Shamir Attack** specifically targets the UOV structure by attempting to recover the secret oil space. This was the attack that broke the original balanced Oil and Vinegar scheme.

**Mathematical Foundation:**
- UOV uses a trapdoor: oil variables never multiply with each other
- The oil space `O` is an `m`-dimensional subspace where the quadratic form degenerates
- For vectors in the oil space, the quadratic form becomes linear

**Attack Strategy:**
1. **Find oil vectors**: Look for vectors `v` such that `P(v)` has special properties
2. **Linear algebra approach**: Set up equations that must be satisfied by oil vectors
3. **Probability amplification**: Use the fact that oil vectors are easier to find than random collisions

**Why this broke balanced OV:**
- In balanced OV (`n = 2m`), the attack runs in polynomial time `O(m^4)`
- The equations become linear and easily solvable
- The oil space can be completely recovered

**Why UOV resists this attack:**
- The unbalanced condition `n > 2m` makes the attack exponential
- Complexity becomes `O(q^(n-2m) * poly(m,n))`
- For proper parameters, `q^(n-2m)` is exponentially large
- Additional vinegar variables create enough "noise" to protect the oil space

**Security Parameter Dependence:**
- Critical parameter: `n - 2m` (number of extra vinegar variables)
- Attack complexity: `O(q^(n-2m))`
- For security, need `q^(n-2m)` sufficiently large
- In uov-Ip: `256^(112-88) = 256^24 ≈ 2^192` operations

In [60]:
def KSAttack(public_key_polynomials):
    
    # Extract parameters from public key
    P = public_key_polynomials  
    PR = P[0].parent()        
    Fq = PR.base_ring()        
    n = PR.ngens()             
    m = len(P)                 
    v = n - m                  
    
    # Step 1: For each public polynomial P[l], extract its symmetric matrix representation
    print("Step 1: Extracting symmetric matrices from public polynomials...")
    G = []
    
    vars = PR.gens()
    
    for l in range(m):
        M_tmp = []
        for i in range(n):
            row = []
            for j in range(n):
                if i == j:
                    # Diagonal: coefficient of x_i**2
                    coeff = P[l].monomial_coefficient(vars[i]**2)
                    row.append(coeff)
                else:
                    # Off-diagonal: coefficient of x_i*x_j
                    coeff = P[l].monomial_coefficient(vars[i] * vars[j])
                    row.append(coeff)
            M_tmp.append(row)
        
        # Convert to matrix
        M_matrix = matrix(Fq, M_tmp)
        G.append(M_matrix)
    
    print(f"Extracted {len(G)} symmetric matrices")
    
    # Step 2: Find suitable linear combinations
    print("Step 2: Finding suitable linear combinations...")
    
    max_attempts = 1000
    attempt = 0
    found_suitable = False
    
    while attempt < max_attempts and not found_suitable:
        attempt += 1
        
        # Create random invertible linear combination G_i
        G_i = zero_matrix(Fq, n, n)
        while G_i.determinant() == 0:
            G_i = zero_matrix(Fq, n, n)
            for i in range(m):
                G_i += Fq.random_element() * G[i]
        
        # Create another random linear combination G_j
        G_j = zero_matrix(Fq, n, n)
        for i in range(m):
            G_j += Fq.random_element() * G[i]
        
        # Compute G_ij = G_i**-1 * G_j
        try:
            G_ij = G_i.inverse() * G_j
        except:
            continue  # Skip if inversion fails
        
        # Step 3: Analyze characteristic polynomial
        f = G_ij.charpoly()
        
        # Check if f is a square (has only squared factors)
        fac = f.factor()
        pwr_sum = sum(factor[1] for factor in fac)
        
        # We want exactly 2 as the sum of exponents (f = g**2)
        if pwr_sum == 2:
            print(f"Found suitable combination after {attempt} attempts")
            found_suitable = True
            break
    
    if not found_suitable:
        print(f"Failed to find suitable combination after {max_attempts} attempts")
        return None
    
    # Step 4: Extract the polynomial and compute kernel
    print("Step 4: Computing kernel of polynomial evaluation...")
    
    # Take the first factor (should be the square root)
    Poly_x = fac[0][0]  # This is g(x) such that f(x) = g(x)**2
    
    # Evaluate the polynomial at G_ij: A = g(G_ij)
    A = Poly_x(G_ij)
    
    # Step 5: Find kernel and basis
    print("Step 5: Finding oil space basis...")
    
    # Kernel of A gives us part of the oil space
    L = A.transpose().kernel().matrix()
    
    # Get basis vectors from kernel
    V = VectorSpace(Fq, n)
    basis = [V(row) for row in L.rows()]
    
    print(f"Found {len(basis)} basis vectors in kernel")
    
    # Step 6: Recover transformation matrix T
    print("Step 6: Recovering transformation matrix T...")
    
    # Extend to full basis of F_q**n
    full_basis = list(V.basis())
    
    # For balanced case: we assume oil space dimension = m
    k = n // 2  # For balanced case
    
    # Create basis sequence and reorder
    if len(basis) >= k:
        # Use the kernel basis for oil space
        oil_basis = basis[:k]
        # Find complementary basis for vinegar space
        complement_basis = []
        current_basis = oil_basis.copy()
        
        for vec in full_basis:
            if len(complement_basis) >= k:
                break
            temp_basis = current_basis + [vec]
            temp_matrix = matrix(Fq, temp_basis)
            if temp_matrix.rank() == len(temp_basis):
                complement_basis.append(vec)
                current_basis = temp_basis
        
        # Combine: vinegar first, then oil
        T_rows = complement_basis + oil_basis
    else:
        # Fallback: use simple reordering
        T_rows = full_basis[k:] + full_basis[:k]
    
    # Create transformation matrix
    T = matrix(Fq, T_rows).transpose()
    
    # Return T**-1
    try:
        T_inv = T.inverse()
        print("Transformation matrix T**-1 recovered successfully")
        print(f"T**-1 dimensions: {T_inv.nrows()} x {T_inv.ncols()}")
        return T_inv
    except:
        print("Failed to invert transformation matrix T")
        return None

In [61]:
def extract_public_polynomials(uov_scheme):
    """
    Extract public key polynomials in format needed for KS attack
    Converts the matrix representation back to polynomial form
    """
    F_q = uov_scheme.F_q
    n = uov_scheme.n
    m = uov_scheme.m
    
    # Create polynomial ring
    PR = PolynomialRing(F_q, n, 'x')
    vars = PR.gens()
    
    public_polys = []
    
    for P_matrix in uov_scheme.expanded_pk:
        poly = PR(0)
        
        # Reconstruct polynomial from symmetric matrix: x^T P x
        for i in range(n):
            for j in range(i, n):
                coeff = P_matrix[i, j]
                if coeff != 0:
                    if i == j:
                        poly += coeff * vars[i]**2  # Fixed: ** instead of ^
                    else:
                        poly += coeff * vars[i] * vars[j]
        
        public_polys.append(poly)
    
    print(f"Extracted {len(public_polys)} public polynomials")
    return public_polys

def demonstrate_ks_attack():
    print("KIPNIS-SHAMIR ATTACK DEMONSTRATION")
    print("=" * 50)
    
    # Use weak balanced parameters for demonstration
    n, m, q = 16, 8, 11
    
    print(f"Using weak parameters: n={n}, m={m}, q={q}")
    print("Note: n = 2m (balanced) - vulnerable to KS attack")
    
    # Create UOV instance with weak parameters
    uov = UOVSignatureScheme(n, m, q, allow_weak=True)
    
    # Extract public key polynomials
    public_polys = extract_public_polynomials(uov)
    
    # Run KS attack
    T_recovered = KSAttack(public_polys)
    
    if T_recovered is not None:
        print("\n KS ATTACK SUCCESSFUL!")
        print("The secret transformation T has been recovered")
        print("This demonstrates the vulnerability of balanced Oil & Vinegar")
    else:
        print("\n KS attack failed")
        print("This might indicate implementation issues or stronger parameters")
    
    return T_recovered

# Example usage:
if __name__ == "__main__":
    # Run the attack demonstration
    T_result = demonstrate_ks_attack()

KIPNIS-SHAMIR ATTACK DEMONSTRATION
Using weak parameters: n=16, m=8, q=11
Note: n = 2m (balanced) - vulnerable to KS attack
⚠️  WARNING: Using weak parameters n=16, m=8 (n ≤ 2m)
   This is only for attack demonstration - INSECURE for real use!
Extracted 8 public polynomials
Step 1: Extracting symmetric matrices from public polynomials...
Extracted 8 symmetric matrices
Step 2: Finding suitable linear combinations...
Found suitable combination after 2 attempts
Step 4: Computing kernel of polynomial evaluation...
Step 5: Finding oil space basis...
Found 1 basis vectors in kernel
Step 6: Recovering transformation matrix T...
Transformation matrix T**-1 recovered successfully
T**-1 dimensions: 16 x 16

 KS ATTACK SUCCESSFUL!
The secret transformation T has been recovered
This demonstrates the vulnerability of balanced Oil & Vinegar


In [62]:
def verify_ks_attack_success(uov_scheme, T_recovered, test_vectors=3):
    F_q = uov_scheme.F_q
    n = uov_scheme.n
    m = uov_scheme.m
    o = n - m
    
    print("Verifying KS attack success...")
    
    # Test 1: Oil space property
    actual_O_bar = uov_scheme.O_bar
    oil_vectors = [actual_O_bar.column(i) for i in range(min(2, m))]
    
    oil_zero_count = 0
    for oil_vec in oil_vectors:
        transformed_oil = T_recovered * oil_vec
        result = Eval(uov_scheme.expanded_pk, transformed_oil)
        if all(r == 0 for r in result):
            oil_zero_count += 1
    
    print(f"Oil space test: {oil_zero_count}/{len(oil_vectors)} correct")
    
    # Test 2: Structure patterns
    structure_correct = 0
    for test in range(test_vectors):
        vinegar_only = vector(F_q, [F_q.random_element() for _ in range(o)] + [0]*m)
        transformed = T_recovered * vinegar_only
        result = Eval(uov_scheme.expanded_pk, transformed)
        if any(result):
            structure_correct += 1
    
    print(f"Structure test: {structure_correct}/{test_vectors} correct")
    
    # Test 3: Signature transformation
    try:
        message = "Test message"
        signature, salt, attempts = uov_scheme.sign(message)
        s = signature[0]
        s_transformed = T_recovered * s
        print(f"Signature transformation: successful")
        sig_test = True
    except:
        print(f"Signature transformation: failed")
        sig_test = False
    
    # Final assessment
    verification_score = 0
    if oil_zero_count >= 1:
        verification_score += 1
    if structure_correct >= test_vectors - 1:
        verification_score += 1
    if sig_test:
        verification_score += 1
    
    print(f"Verification score: {verification_score}/3")
    
    if verification_score >= 2:
        print("KS ATTACK SUCCESSFUL - UOV is broken")
        return True
    else:
        print("KS ATTACK FAILED - transformation not correct")
        return False

def test_forgery_capability(uov_scheme, T_recovered):
    F_q = uov_scheme.F_q
    n = uov_scheme.n
    m = uov_scheme.m
    o = n - m
    
    target = vector(F_q, [F_q.random_element() for _ in range(m)])
    
    for attempt in range(50):
        vinegar = vector(F_q, [F_q.random_element() for _ in range(o)])
        oil = vector(F_q, [F_q.random_element() for _ in range(m)])
        s_transformed = vector(list(vinegar) + list(oil))
        s_original = T_recovered.inverse() * s_transformed
        result = Eval(uov_scheme.expanded_pk, s_original)
        if result == target:
            return True
    return False

def demonstrate_complete_verification():
    print("KS ATTACK WITH VERIFICATION")
    print("=" * 40)
    
    n, m, q = 16, 8, 11
    print(f"Parameters: n={n}, m={m}, q={q}")
    
    uov = UOVSignatureScheme(n, m, q, allow_weak=True)
    public_polys = extract_public_polynomials(uov)
    T_recovered = KSAttack(public_polys)
    
    if T_recovered is not None:
        print("Verifying attack results...")
        is_successful = verify_ks_attack_success(uov, T_recovered)
        
        if is_successful:
            print("CONCLUSION: KS attack successful - scheme broken")
        else:
            print("CONCLUSION: KS attack partially successful")
    else:
        print("CONCLUSION: KS attack failed")

def demonstrate_ks_attack():
    demonstrate_complete_verification()

if __name__ == "__main__":
    demonstrate_ks_attack()

KS ATTACK WITH VERIFICATION
Parameters: n=16, m=8, q=11
⚠️  WARNING: Using weak parameters n=16, m=8 (n ≤ 2m)
   This is only for attack demonstration - INSECURE for real use!
Extracted 8 public polynomials
Step 1: Extracting symmetric matrices from public polynomials...
Extracted 8 symmetric matrices
Step 2: Finding suitable linear combinations...
Found suitable combination after 6 attempts
Step 4: Computing kernel of polynomial evaluation...
Step 5: Finding oil space basis...
Found 7 basis vectors in kernel
Step 6: Recovering transformation matrix T...
Transformation matrix T**-1 recovered successfully
T**-1 dimensions: 16 x 16
Verifying attack results...
Verifying KS attack success...
Oil space test: 0/2 correct
Structure test: 3/3 correct
Signature transformation: failed
Verification score: 1/3
KS ATTACK FAILED - transformation not correct
CONCLUSION: KS attack partially successful
